In [136]:
import matplotlib.pyplot as plt
import librosa
from IPython.display import Audio
import pandas as pd
import numpy as np
import torch
from bc import bit_cypher



In [2]:
path = [f'../data/mp3_train_files/Gould/Gould - WTC_clip_{x}.mp3' for x in range(1,10)] + [f'../data/mp3_train_files/Schiff/Schiff - WTC_clip_{x}.mp3' for x in range(1,10)]
x = [librosa.load(x,sr=None)[0] for x in path]
x = torch.stack([torch.tensor(x) for x in x])
train = x[0:9]
test = x[9:]

sr = 44100
e = 3

In [3]:
x.shape

torch.Size([18, 220500])

In [4]:
pd.Series(x.flatten()).nunique()

3799926

In [5]:
x.min(),x.max()

(tensor(-0.1970), tensor(0.2072))

In [6]:
Audio(x[13],rate=sr)

In [7]:
test[0].squeeze().shape

torch.Size([220500])

In [98]:
from sklearn.cluster import KMeans


kmeans = KMeans(n_clusters=511)
kmeans.fit(train[0:5].reshape(-1, 1))


KMeans(n_clusters=511)

In [99]:
idx = torch.stack(([torch.tensor(kmeans.predict(t.reshape(-1,1))) for t in test])) # idxs
 
centroids = kmeans.cluster_centers_.flatten() # vocab

quantized_data = centroids[idx] 

In [100]:
Audio(quantized_data[0],rate=sr)

In [112]:
centroid2embed = {x:i for i,x in enumerate(pd.Series(idx.flatten()).value_counts().index.to_list())}

In [114]:
centroid2embed[201]

0

In [115]:
u,v,vocab_size = bit_cypher(9)

In [130]:
idx[5]

tensor([201, 201, 201,  ..., 297, 382,   3], dtype=torch.int32)

In [132]:
embedding_idx = [centroid2embed[int(x)] for x in idx[5]]

In [133]:
embeddings = v[embedding_idx]

In [135]:
embeddings

torch.Size([220500, 9])